In [1]:
from app.rl.tictactoe.t3_test import T3Model, convert_inputs
from gym_tic_tac_toe.envs.tic_tac_toe_env import TicTacToeEnv, Pix
import torch
from torch.nn import functional as F
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
env = TicTacToeEnv()
env.reset()
env.step(0)
env.step(1)
env.step(2)
env.step(5)
env.render()

XOX
__O
___



In [4]:
model = T3Model(8).double().to(device)

In [5]:
x = convert_inputs(env.state, env.player)
x = torch.stack((x, x))
x.T.shape

torch.Size([9, 2])

In [6]:
y = model(x)
y.shape

torch.Size([2, 9])

In [ ]:
def get_loss(probs, rewards, is_dones):
    pass


model.train()
optim = torch.optim.Adam(model.parameters(), lr=1e-2)
batch_size = 2
envs = [TicTacToeEnv() for _ in range(batch_size)]
losses = []

for epoch in range(10):

    rewards = []
    probs = []
    is_dones = []

    # Monte Carlo loop
    while True:

        states = [env.reset() for env in envs]
        xs = [convert_inputs(states[i], envs[i].player) for i in range(batch_size)]
        xs = torch.tensor(xs).double().to(device)
        legal_actions = [env.get_legal_actions() for env in envs]

        yh = model(xs)  # yh shape: (batch, 9)

        action_idx = []
        rewards_t = []
        is_dones_t = []
        for i in range(batch_size):
            if is_dones[-1][i]:
                action_i, reward, is_done = None, None, None
            else:
                action_i = int(torch.multinomial(yh[i, legal_actions[i]], 1).squeeze())
                state, reward, is_done, info = env.step(action_i)

            action_idx.append(action_i)
            rewards_t.append(reward)
            is_dones_t.append(is_done)

        probs.append(yh[range(batch_size), action_idx])
        rewards.append(rewards_t)
        is_dones.append(is_dones_t)
        
        if all(is_dones_t):
            break
        
    loss = get_loss(probs, rewards, is_dones)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    losses.append(loss.item())

In [7]:
la = env.get_legal_actions()
la

array([3, 4, 6, 7, 8])

In [8]:
y.shape, la.shape

(torch.Size([2, 9]), (5,))

In [9]:
y[0]

tensor([0.0200, 0.0341, 0.0000, 0.0000, 0.0000, 0.0646, 0.0000, 0.0181, 0.0000],
       dtype=torch.float64, grad_fn=<SelectBackward>)

In [14]:
idx=torch.multinomial(y[0], 1)
idx

tensor([1])

In [15]:
y[0, idx]

tensor([0.0341], dtype=torch.float64, grad_fn=<IndexBackward>)